Fazendo a primeira chamada da LLM

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [6]:
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash', google_api_key=os.getenv("GOOGLE_API_KEY"))

In [7]:
llm.invoke('Quem foi Albert Einstein?').content

'Albert Einstein (1879-1955) foi um físico teórico alemão que desenvolveu a teoria da relatividade, uma das duas teorias pilares da física moderna (a outra é a mecânica quântica).  Sua obra é também conhecida por sua influência na filosofia da ciência.\n\nEle é mais conhecido pela equação de massa-energia, E=mc², que afirma que a energia (E) e a massa (m) são equivalentes e podem ser convertidas uma na outra, com c representando a velocidade da luz.  Essa equação revolucionou a nossa compreensão da energia nuclear.\n\nAlém da relatividade, Einstein fez contribuições significativas em outras áreas da física, incluindo:\n\n* **Efeito fotoelétrico:** Explicou o efeito fotoelétrico, mostrando a natureza quântica da luz, o que lhe rendeu o Prêmio Nobel de Física em 1921.\n* **Movimento browniano:** Contribuiu para a compreensão do movimento browniano, fornecendo evidências experimentais para a existência de átomos e moléculas.\n* **Teoria do campo unificado:** Tentou, sem sucesso, unificar 